<a href="https://colab.research.google.com/github/ariakhademi/telematics/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install opendatasets

In [8]:
import opendatasets as od
import pandas as pd

dataset_url = 'https://www.kaggle.com/datasets/yunlevin/levin-vehicle-telematics'
od.download(dataset_url)
file_path = 'levin-vehicle-telematics/allcars.csv'

# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_path)

Skipping, found downloaded files in "./levin-vehicle-telematics" (use force=True to force download)


<ipython-input-8-07865fba3548>:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [9]:
print(df.head(10))

   Unnamed: 0                   timeStamp  tripID  \
0           0  2017-10-31 18:45:59.000000       0   
1           1  2017-10-31 18:46:00.000000       0   
2           2  2017-10-31 18:46:01.000000       0   
3           3  2017-10-31 18:46:02.000000       0   
4           4  2017-10-31 18:46:03.000000       0   
5           5  2017-10-31 18:46:04.000000       0   
6           6  2017-10-31 18:46:05.000000       0   
7           7  2017-10-31 18:46:06.000000       0   
8           8  2017-10-31 18:46:07.000000       0   
9           9  2017-10-31 18:46:08.000000       0   

                                             accData  gps_speed  battery  \
0  ffb0fd700220fff43fffe74808e73e03f33400ef33fbe1...        2.2    14.48   
1  00b0fda802e804e14104ed3d01ee3a08ec4c04f738fbd5...        3.7    14.48   
2  00f0fe5002b0fdea4009e33c01ed3701ec4100f53b05e8...        3.9    14.54   
3  0120fdc0028804e03c10de3a09e8400ce43b0ae63b01f4...        3.9    14.56   
4  00a0fde802d816e0370ae73604e23b04e

Detect hard brakes

In [14]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])
print(df.head(10))

   Unnamed: 0           timeStamp  tripID  \
0           0 2017-10-31 18:45:59       0   
1           1 2017-10-31 18:46:00       0   
2           2 2017-10-31 18:46:01       0   
3           3 2017-10-31 18:46:02       0   
4           4 2017-10-31 18:46:03       0   
5           5 2017-10-31 18:46:04       0   
6           6 2017-10-31 18:46:05       0   
7           7 2017-10-31 18:46:06       0   
8           8 2017-10-31 18:46:07       0   
9           9 2017-10-31 18:46:08       0   

                                             accData  gps_speed  battery  \
0  ffb0fd700220fff43fffe74808e73e03f33400ef33fbe1...        2.2    14.48   
1  00b0fda802e804e14104ed3d01ee3a08ec4c04f738fbd5...        3.7    14.48   
2  00f0fe5002b0fdea4009e33c01ed3701ec4100f53b05e8...        3.9    14.54   
3  0120fdc0028804e03c10de3a09e8400ce43b0ae63b01f4...        3.9    14.56   
4  00a0fde802d816e0370ae73604e23b04e53900e542ffea...        4.7    14.48   
5  0010fd5802c012e2430e013303ea45fdf03a08e2340be

In [15]:
acceleration_threshold = 0.5

delta_speed = df['speed'].diff()
delta_time = df['timeStamp'].diff().dt.total_seconds()
acceleration = delta_speed / delta_time
df['acceleration'] = acceleration
df['hard_brakes'] = (df['acceleration'] < -acceleration_threshold).astype(int)
print(df['hard_brakes'].head(10))

0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    0
8    1
9    1
Name: hard_brakes, dtype: int64
